In [19]:
import numpy as np
import os
import re
import sys

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [11]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# remove sonnet numbers and convert to lowercase
text = re.sub(r'[0-9]+', '', text) 
text = text.lower()

# create mapping of unique chars to integers as well as reverse mapping
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(text) # total length of dataset
n_vocab = len(chars) # number of unique characters

In [12]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_sequences = len(dataX)

In [13]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [14]:
# define the LSTM model 
# (single layer with 150 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [15]:
# load the network weights from best checkpoint
filename = "weights-improvement-15-2.3879.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [23]:
# generate emissions

# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  love thee with mine eyes,
for they in t "
he whel the sore the sore,
                    
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee,

                   
the whet the tore the sore the sore to toee